## B-anthracis-pXO1-analysis
#### Identifying DNA processing and replication related domains in proteins encoded in B.anthracis pXO1 Plasmids (Section 2.9, 3.7) 

In [1]:
import json
import glob
from collections import defaultdict

## General
from __future__ import annotations
from pathlib import Path
from collections import defaultdict
from typing import DefaultDict, Dict, List
import json

### Step 1: Load reference domain and superfamilies from sequences in reference STRING DB network
##### Reference domains from reference B.cereus and B.anthracis NERD-domain interaction networks

In [4]:
# === Load domain hits per reference sequence from JSON files ===
domain_annotations: Dict[str, str] = {}
domains_to_seqs: DefaultDict[str, List[str]] = defaultdict(list)
ref_domains: List[str] = []

domain_dir = Path("../files/domain_search_files/b_anthracis_reference_seq_domain_json")

for file_path in domain_dir.glob("*.json"):
    ref_seq: str = file_path.name.split("_")[0]
    print(ref_seq)

    with file_path.open("r") as f:
        records = json.load(f)

    # Each record expected to have "metadata" with "accession" and "name"
    for rec in records:
        md = rec.get("metadata", {})
        accession = md.get("accession")
        name = md.get("name")

        if not accession:
            continue

        domains_to_seqs[ref_seq].append(accession)

        if name:
            domain_annotations[accession] = name

# === Parse reference network metadata and extend domain mappings ===

ref_meta_path = Path(
    "../files/domain_search_files/StringDB_NERD_reference_network_metadata.json"
)

with ref_meta_path.open("r") as f:
    ref_meta = json.load(f)

for res in ref_meta.get("results", []):
    # Expect something like: "xref": [{"id": "<sequence_id>", ...}]
    xref_list = res.get("xref", [])
    xref_id = xref_list[0].get("id") if xref_list and isinstance(xref_list[0], dict) else None

    for match in res.get("matches", []):
        # Keep only matches that contain an e-value and are Domains / Homologous Superfamilies
        if "evalue" not in match:
            continue

        signature = match.get("signature") or {}
        if signature.get("type") not in {"DOMAIN", "HOMOLOGOUS_SUPERFAMILY"}:
            continue

        entry = signature.get("entry")
        if not entry:
            continue

        accession = entry.get("accession")
        name = entry.get("name")
        if not accession:
            continue

        ref_domains.append(accession)

        if name:
            domain_annotations[accession] = name

        if xref_id:
            domains_to_seqs[xref_id].append(accession)

# === Breakdown of Reference Domains ===
print(len(domain_annotations), "unique domain annotations")
print(len(domains_to_seqs), "reference sequences mapped")
print(len(ref_domains), "reference domains")


42 unique domain annotations
10 reference sequences mapped
57 reference domains


### Step 2: Map InterProScan Results from pXO1 proteins sequences to domains in reference NERD network
#### InterProScan searches were run directly on: https://www.ebi.ac.uk/interpro/search/sequence/
#### Sequences used in InterProScan search are located at: ./files/domain_search_files/pXO1_protein_InterProScan_searches

In [3]:
# === Map Pfam domains in plasmid sequences to reference sequences and functional annotations ===

# Set up data structures for results
pfam_domain_accessions: List[str] = []
reference_sequences_hit: List[str] = []
sequences_with_hits: List[str] = []
matched_accessions: List[str] = []
domains_by_function: DefaultDict[str, Set[Tuple[str, str, str]]] = defaultdict(set)

# === Search InterProScan Results of pXO1 plasmid sequences to find matches in reference network domains ===
for file_index in range(1, 4):
    file_path = f"../files/domain_search_files/pXO1_plasmid_domain_searches/domain_search_{file_index}.json"

    with open(file_path, "r") as f:
        search_data = json.load(f)

    for result in search_data.get("results", []):
        for match in result.get("matches", []):
            if "evalue" in match and match["signature"].get("entry"):
                accession = match["signature"]["entry"].get("accession")
                if not accession:
                    continue

                pfam_domain_accessions.append(accession)

                if accession in ref_domains:
                    ref_seq_id = next(
                        k for k, v in domains_to_seqs.items() if accession in v
                    )

                    matched_accessions.append(accession)
                    reference_sequences_hit.append(ref_seq_id)
                    sequences_with_hits.append(result["xref"][0]["id"])
                    


In [8]:
# === Summary of results ===
print("\n--- Quick Summary ---")
print(f"Total Pfam domain accessions found: {len(pfam_domain_accessions)}")
print(f"Unique Pfam domain accessions: {len(set(pfam_domain_accessions))}")
print(f"Matched accessions: {len(matched_accessions)}")
print(f"Reference sequences hit: {len(set(reference_sequences_hit))}")
print(f"Sequences with hits: {len(set(sequences_with_hits))}")

# Show first few entries
print("\nSample Pfam domain accessions:", pfam_domain_accessions[:5])
print("Sample matched accessions:", matched_accessions[:5])
print("Sample sequences with hits:", sequences_with_hits[:5])

# Sample domains by one function
if domains_by_function:
    first_func = next(iter(domains_by_function))
    print(f"\nExample domains in function '{first_func}':")
    for domain_tuple in list(domains_by_function[first_func])[:5]:
        print(domain_tuple)



--- Quick Summary ---
Total Pfam domain accessions found: 519
Unique Pfam domain accessions: 200
Matched accessions: 53
Reference sequences hit: 5
Sequences with hits: 31

Sample Pfam domain accessions: ['IPR023346', 'IPR011055', 'IPR050570', 'IPR011055', 'IPR016047']
Sample matched accessions: ['IPR043519', 'IPR043519', 'IPR019734', 'IPR011990', 'IPR011990']
Sample sequences with hits: ['WP_000581648.1', 'WP_000581648.1', 'WP_000637338.1', 'WP_000637338.1', 'WP_000637338.1']
